In [1]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
inputs = torch.randn(1, 3, 300, 300)

In [ ]:
model = torch.jit.load("path_to_model").eval()

In [29]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

STAGE:2024-11-20 10:19:31 75929:75929 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-11-20 10:19:31 75929:75929 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-11-20 10:19:31 75929:75929 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [30]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=100))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         2.06%     310.000us       100.00%      15.031ms      15.031ms             1  
                          forward         3.35%     504.000us        97.94%      14.721ms      14.721ms             1  
                quantized::conv2d        49.67%       7.466ms        52.85%       7.944ms      92.372us            86  
                      aten::relu_        32.51%       4.886ms        32.51%       4.886ms     113.628us            43  
                        aten::cat         1.58%     237.000us         3.18%     478.000us      79.667us             6  
                   quantized::add       

In [23]:
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

STAGE:2024-11-20 10:19:14 75929:75929 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-11-20 10:19:14 75929:75929 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-11-20 10:19:14 75929:75929 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [24]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=100))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.38%      59.000us         0.38%      59.000us       1.405us      13.74 Mb      13.74 Mb            42  
    aten::_empty_affine_quantized         1.45%     227.000us         1.45%     227.000us       2.121us      13.57 Mb      13.57 Mb           107  
        aten::quantize_per_tensor         0.89%     139.000us         0.89%     139.000us      46.333us     336.91 Kb     336.91 Kb             3  
                   aten::_softmax         0.54%      85.000us         0.54%      85.000us      85.000us     246.